In [1]:
import numpy as np
from scipy.optimize import fmin
from astropy.stats import bootstrap
from random import shuffle
import random
import time
from astropy.stats import bootstrap

%matplotlib inline
import matplotlib.pyplot as plt

In [8]:
maze_ori = np.matrix("99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99;\
                 99, 0, 1, 0, 2, 0, 8, 0, 5, 0, 3, 0, 1, 0, 1, 0, 99;\
           99, -2, 99, 2, 99, 10, 99, 5, 99, -1, 99, 5, 99, 10, 99, -1, 99;\
           99, 0, 2, 0, 7, 0, -1, 0, 2, 0, 2, 0, 3, 0, 5, 0, 99;\
           99, 7, 99, 5, 99, 2, 99, -1, 99 2, 99, 3, 99, 8, 99, 1, 99;\
           99, 0, 3, 0, 2, 0, 6, 0, 4, 0, 2, 0, 1, 0, 8, 0, 99;\
           99, 4, 99, 6, 99, -1, 99, 2, 99, -1, 99, 8, 99, -1, 99, 8, 99;\
           99, 0, 2, 0, 5, 0, 3, 0, 4, 0, 1, 0, -1, 0, 3, 0, 99;\
           99, 2, 99, 10, 99, 4, 99, 1, 99, 9, 99, 4, 99, 20, 99, 5, 99;\
           99, 0, -2, 0, 3, 0, 4, 0, 7, 0, -1, 0, 1, 0, 3, 0, 99;\
           99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99")
maze_ori = maze_ori.tolist()

# dirs=[(0,1),(1,0),(0,-1),(-1,0)]
path=[]

def mark(maze,pos):  #给迷宫maze的位置pos标"2"表示“倒过了”
    if maze[pos[0]][pos[1]] == 0:
        maze[pos[0]][pos[1]]=0
    else:
        maze[pos[0]][pos[1]]=99
    return maze

def passable(maze,pos): #检查迷宫maze的位置pos是否可通行
    return maze[pos[0]][pos[1]]!=99


def find_path(maze,pos,end,last_step=(1,0)):
    mark(maze,pos)
    if last_step == (1,0):
        dirs = [(0,1),(1,0),(0,-1)]
    if last_step == (-1,0):
        dirs = [(0,1),(0,-1),(-1,0)]
    if last_step == (0,1):
        dirs = [(0,1),(1,0),(-1,0)]
    if last_step == (0,-1):
        dirs = [(1,0),(0,-1),(-1,0)]
    random.sample(dirs, len(dirs))
    dirs = sorted(dirs, key=lambda k: random.random())
    if pos==end:
#         print(pos,end=" ")  #已到达出口，输出这个位置。成功结束
        path.append(pos)
        return True
    for i in range(3):      #否则按四个方向顺序检查
        nextp = pos[0]+dirs[i][0],pos[1]+dirs[i][1]
        last_step = (dirs[i][0],dirs[i][1])
        #考虑下一个可能方向
        if passable(maze,nextp):        #不可行的相邻位置不管
            if find_path(maze,nextp,end,last_step):#如果从nextp可达出口，输出这个位置，成功结束
#                 print(pos,end=" ")
                path.append(pos)
                return True
    return False


def see_path(maze,path):     #使寻找到的路径可视化
    for i,p in enumerate(path):
        if i==0:
            maze[p[0]][p[1]] ="E"
        elif i==len(path)-1:
            maze[p[0]][p[1]]="S"
        else:
            maze[p[0]][p[1]] =88
    print("\n")
    for r in maze:
        for c in r:
            if c==88:
                print('\033[0;31m'+"*"+" "+'\033[0m',end="")
            elif c=="S" or c=="E":
                print('\033[0;34m'+c+" " + '\033[0m', end="")
            else:
                print(" "*2,end="")
        print()

def map_value(maze,path):
    begin = 1 
    path = list(reversed(path))
    for i,p in enumerate(path):
        if maze[p[0]][p[1]] == 10:
            begin *= 2
        elif maze[p[0]][p[1]] == 20:
            begin *= 3
        else:
            begin += maze[p[0]][p[1]]
#         print("+ %s = %s" %(maze[p[0]][p[1]],begin))
    return begin
        
value  = []
allpath = []
for i in range(500000):
    maze = [[99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99],\
            [99, 0, 1, 0, 2, 0, 8, 0, 5, 0, 3, 0, 1, 0, 1, 0, 99],\
            [99, -2, 99, 2, 99, 10, 99, 5, 99, -1, 99, 5, 99, 10, 99, -1, 99],\
            [99, 0, 2, 0, 7, 0, -1, 0, 2, 0, 2, 0, 3, 0, 5, 0, 99],\
            [99, 7, 99, 5, 99, 2, 99, -1, 99, 2, 99, 3, 99, 8, 99, 1, 99],\
            [99, 0, 3, 0, 2, 0, 6, 0, 4, 0, 2, 0, 1, 0, 8, 0, 99],\
            [99, 4, 99, 6, 99, -1, 99, 2, 99, -1, 99, 8, 99, -1, 99, 8, 99],\
            [99, 0, 2, 0, 5, 0, 3, 0, 4, 0, 1, 0, -1, 0, 3, 0, 99],\
            [99, 2, 99, 10, 99, 4, 99, 1, 99, 9, 99, 4, 99, 20, 99, 5, 99],\
            [99, 0, -2, 0, 3, 0, 4, 0, 7, 0, -1, 0, 1, 0, 3, 0, 99],\
            [99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99]]
    start=(9,1)
    end=(1,15)
    path=[]
    find_path(maze,start,end)
    allpath.append(path)
    value.append(map_value(maze_ori,path))

In [9]:
maxval = np.max(value)
maxpos = np.argmax(value)

print(maxval)
print(list(reversed(allpath[maxpos])))
see_path(maze,allpath[maxpos])

1545
[(9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (8, 5), (7, 5), (7, 4), (7, 3), (6, 3), (5, 3), (5, 4), (5, 5), (6, 5), (7, 5), (7, 6), (7, 7), (8, 7), (9, 7), (9, 8), (9, 9), (8, 9), (7, 9), (7, 8), (7, 7), (6, 7), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11), (5, 12), (5, 13), (5, 14), (5, 15), (6, 15), (7, 15), (7, 14), (7, 13), (7, 12), (7, 11), (6, 11), (5, 11), (4, 11), (3, 11), (3, 10), (3, 9), (3, 8), (3, 7), (4, 7), (5, 7), (5, 6), (5, 5), (4, 5), (3, 5), (3, 4), (3, 3), (3, 2), (3, 1), (4, 1), (5, 1), (5, 2), (5, 3), (4, 3), (3, 3), (2, 3), (1, 3), (1, 4), (1, 5), (2, 5), (3, 5), (3, 6), (3, 7), (2, 7), (1, 7), (1, 8), (1, 9), (2, 9), (3, 9), (4, 9), (5, 9), (6, 9), (7, 9), (7, 10), (7, 11), (8, 11), (9, 11), (9, 12), (9, 13), (8, 13), (7, 13), (6, 13), (5, 13), (4, 13), (3, 13), (2, 13), (1, 13), (1, 12), (1, 11), (2, 11), (3, 11), (3, 12), (3, 13), (3, 14), (3, 15), (2, 15), (1, 15)]


                                  
      * * *   * * *   * * *   E   
      *   *   *   *   *  

In [10]:
maze_ori

[[99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99],
 [99, 0, 1, 0, 2, 0, 8, 0, 5, 0, 3, 0, 1, 0, 1, 0, 99],
 [99, -2, 99, 2, 99, 10, 99, 5, 99, -1, 99, 5, 99, 10, 99, -1, 99],
 [99, 0, 2, 0, 7, 0, -1, 0, 2, 0, 2, 0, 3, 0, 5, 0, 99],
 [99, 7, 99, 5, 99, 2, 99, -1, 99, 2, 99, 3, 99, 8, 99, 1, 99],
 [99, 0, 3, 0, 2, 0, 6, 0, 4, 0, 2, 0, 1, 0, 8, 0, 99],
 [99, 4, 99, 6, 99, -1, 99, 2, 99, -1, 99, 8, 99, -1, 99, 8, 99],
 [99, 0, 2, 0, 5, 0, 3, 0, 4, 0, 1, 0, -1, 0, 3, 0, 99],
 [99, 2, 99, 10, 99, 4, 99, 1, 99, 9, 99, 4, 99, 20, 99, 5, 99],
 [99, 0, -2, 0, 3, 0, 4, 0, 7, 0, -1, 0, 1, 0, 3, 0, 99],
 [99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99]]

In [11]:
def map_value(maze,path):
    begin = 1 
    path = list(reversed(path))
    for i,p in enumerate(path):
        if maze[p[0]][p[1]] == 10:
            begin *= 2
        elif maze[p[0]][p[1]] == 20:
            begin *= 3
        else:
            begin += maze[p[0]][p[1]]
        if maze[p[0]][p[1]] == 10:
            print("* 2 = %s" %(begin))
        elif maze[p[0]][p[1]] == 20:
            print("* 3 = %s" %(begin))
        elif maze[p[0]][p[1]] != 0:
            print("+ %s = %s" %(maze[p[0]][p[1]],begin))
    return begin

print("start at 1")      
map_value(maze_ori,allpath[maxpos])
# see_path(maze,allpath[maxpos])

start at 1
+ -2 = -1
+ 3 = 2
+ 4 = 6
+ 5 = 11
+ 6 = 17
+ 2 = 19
+ -1 = 18
+ 3 = 21
+ 1 = 22
+ 7 = 29
+ 9 = 38
+ 4 = 42
+ 2 = 44
+ 4 = 48
+ 2 = 50
+ 1 = 51
+ 8 = 59
+ 8 = 67
+ 3 = 70
+ -1 = 69
+ 8 = 77
+ 3 = 80
+ 2 = 82
+ 2 = 84
+ -1 = 83
+ 6 = 89
+ 2 = 91
+ 7 = 98
+ 2 = 100
+ 7 = 107
+ 3 = 110
+ 5 = 115
+ 2 = 117
+ 2 = 119
* 2 = 238
+ -1 = 237
+ 5 = 242
+ 5 = 247
+ -1 = 246
+ 2 = 248
+ -1 = 247
+ 1 = 248
+ 4 = 252
+ 1 = 253
* 3 = 759
+ -1 = 758
+ 8 = 766
* 2 = 1532
+ 1 = 1533
+ 5 = 1538
+ 3 = 1541
+ 5 = 1546
+ -1 = 1545


1545

In [12]:
maze_ori = np.matrix("99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99;\
                 99, 0, 1, 0, 2, 0, 8, 0, 5, 0, 3, 0, 1, 0, 1, 0, 99;\
           99, -2, 99, 2, 99, 10, 99, 5, 99, -1, 99, 5, 99, 10, 99, -1, 99;\
           99, 0, 2, 0, 7, 0, -1, 0, 2, 0, 2, 0, 3, 0, 5, 0, 99;\
           99, 7, 99, 5, 99, 2, 99, -1, 99 2, 99, 3, 99, 8, 99, 1, 99;\
           99, 0, 3, 0, 2, 0, 6, 0, 4, 0, 2, 0, 1, 0, 8, 0, 99;\
           99, 4, 99, 6, 99, -1, 99, 2, 99, -1, 99, 8, 99, -1, 99, 8, 99;\
           99, 0, 2, 0, 5, 0, 3, 0, 4, 0, 1, 0, -1, 0, 3, 0, 99;\
           99, 2, 99, 10, 99, 4, 99, 1, 99, 9, 99, 4, 99, 20, 99, 5, 99;\
           99, 0, -2, 0, 3, 0, 4, 0, 7, 0, -1, 0, 1, 0, 3, 0, 99;\
           99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99 ,99, 99, 99, 99, 99")
print(maze_ori)

[[99 99 99 99 99 99 99 99 99 99 99 99 99 99 99 99 99]
 [99  0  1  0  2  0  8  0  5  0  3  0  1  0  1  0 99]
 [99 -2 99  2 99 10 99  5 99 -1 99  5 99 10 99 -1 99]
 [99  0  2  0  7  0 -1  0  2  0  2  0  3  0  5  0 99]
 [99  7 99  5 99  2 99 -1 99  2 99  3 99  8 99  1 99]
 [99  0  3  0  2  0  6  0  4  0  2  0  1  0  8  0 99]
 [99  4 99  6 99 -1 99  2 99 -1 99  8 99 -1 99  8 99]
 [99  0  2  0  5  0  3  0  4  0  1  0 -1  0  3  0 99]
 [99  2 99 10 99  4 99  1 99  9 99  4 99 20 99  5 99]
 [99  0 -2  0  3  0  4  0  7  0 -1  0  1  0  3  0 99]
 [99 99 99 99 99 99 99 99 99 99 99 99 99 99 99 99 99]]


In [13]:
# def see_path2(maze,path):     #使寻找到的路径可视化
#     for i,p in enumerate(path):
#         if i==0:
#             maze[p[0]][p[1]] ="E"
#         elif i==len(path)-1:
#             maze[p[0]][p[1]]="S"
#         else:
#             maze[p[0]][p[1]] =88
#         print("\n")
#         for r in maze:
#             for c in r:
#                 if c==88:
#                     print('\033[0;31m'+"*"+" "+'\033[0m',end="")
#                 elif c=="S" or c=="E":
#                     print('\033[0;34m'+c+" " + '\033[0m', end="")
#                 elif c==99:
#                     print('\033[0;;40m'+" "*2+'\033[0m',end="")
#                 else:
#                     print('\033[0;;40m'+" "*2+'\033[0m',end="")
#         #                     print(" "*2,end="")
#             print()
# see_path2(maze_ori,allpath[maxpos])